In [2]:
!pip install dfply
import pandas as pd
import numpy as np
import math
from dfply import *
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import statistics 
import plotly.graph_objects as go
from scipy import stats
from scipy.stats import pearsonr
import plotly.graph_objects as go
import warnings
from scipy.stats import shapiro
from datetime import date, timedelta

     |████████████████████████████████| 614kB 2.7MB/s 


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
players_path = '/content/drive/My Drive/Capstone/Data/Clean/Players_Clean.pkl'
natstats_path = '/content/drive/My Drive/Capstone/Data/Clean/natstats_Clean.pkl'
markval_path = '/content/drive/My Drive/Capstone/Data/Clean/markval_Clean.pkl'

players = pd.read_pickle(players_path)
natstats = pd.read_pickle(natstats_path)
markval = pd.read_pickle(markval_path)

In [0]:
natstats = natstats[natstats.national_club.apply(lambda txt: len(re.findall("\d", txt)) < 1)]

In [0]:
natstats_and_pos = pd.merge(natstats, players[["tm_id","main_field_position"]], on = "tm_id")

In [0]:
gk_natstats = natstats_and_pos[natstats_and_pos.main_field_position == "goalkeepers"]
players_natstats = natstats_and_pos[natstats_and_pos.main_field_position != "goalkeepers"]

In [0]:
def wide_natstats(natstats, gk = False):
  no_mpg = natstats.loc[natstats.attribute != "mpg"]
  mpg = natstats.loc[natstats.attribute == "mpg"]
  natstats_wide = pd.pivot_table(no_mpg, values = 'value', index = ["tm_id","national_club"], 
                              columns = "attribute", aggfunc=np.sum)
  natstats_wide.columns = natstats_wide.columns.astype(list)
  natstats_wide.reset_index(inplace = True, drop = False) 
  if not gk:
    mpg_wide = pd.pivot_table(mpg, values = 'value', index = ["tm_id","national_club"], 
                                columns = "attribute", aggfunc=np.mean)
    mpg_wide.columns = mpg_wide.columns.astype(list)
    mpg_wide.reset_index(inplace = True, drop = False) 
    result = pd.merge(natstats_wide, mpg_wide)
  else:
    result = natstats_wide
  return result

In [0]:
gk_natstats_wide = wide_natstats(gk_natstats, True)
players_natstats_wide = wide_natstats(players_natstats)

In [9]:
players_natstats_wide

attribute,tm_id,national_club,a,g,mp,og,pg,rc,syc,s,soff,son,yc,mpg
0,532,Peru,1,11,4313,0,3,0,1,57,21,5,7,378.333333
1,2865,Switzerland,7,8,8459,0,0,0,1,108,34,6,18,705.200000
2,2938,Austria,0,0,419,0,0,0,0,5,2,0,0,0.000000
3,2989,Peru,10,34,7475,0,1,1,1,93,17,8,18,220.250000
4,2997,Australia,0,0,458,0,0,0,0,14,0,10,0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3642,48725441,Serbia,6,6,3466,0,0,0,0,50,17,14,6,397.250000
3643,48742434,Montenegro,2,8,1771,0,2,0,0,30,11,11,3,368.000000
3644,48747085,Serbia,1,0,1524,0,0,1,0,22,3,5,3,0.000000
3645,48759594,Netherlands,0,0,0,0,0,0,0,0,0,0,0,0.000000


In [0]:
gk_natstats_wide

attribute,tm_id,national_club,app,cs,gc,mp,og,rc,syc,soff,son,yc
0,1667,Switzerland,61,25,63,5075,0,0,0,6,3,3
1,2857,Switzerland,1,0,1,90,0,0,0,0,0,0
2,2882,Austria,3,2,1,270,0,0,0,0,0,1
3,3655,Ireland,2,1,1,18,0,0,0,0,2,0
4,3755,Argentina,2,0,4,180,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
381,438682,Brazil,0,0,0,0,0,0,0,0,0,0
382,442531,Senegal,7,5,2,630,0,0,0,0,0,0
383,463320,Venezuela,0,0,0,0,0,0,0,0,0,0
384,541420,DR Congo,0,0,0,0,0,0,0,0,0,0


In [0]:
players_natstats_wide.to_pickle('/content/drive/My Drive/Capstone/Data/Clean/natstats_detailed.pkl')
gk_natstats_wide.to_pickle('/content/drive/My Drive/Capstone/Data/Clean/gk_natstats_detailed.pkl')

In [0]:
def season_to_date(season):
    date = season
    try:
      end = season.split("/")[1]
      date = "20" + str(end)
    except:
      pass
    return date
def get_age(age_delta):
  result = math.nan
  try:
    result = int(age_delta / timedelta(days=365.2425))
  except:
    pass
  return result  

In [0]:
def wide_stats(stats, position):
  global players
  # stats["date"] = stats.index
  stats.reset_index(inplace = True, drop = True)
  stats_by_pos = pd.merge(stats,
                        players[["tm_id","main_field_position"]],
                        on="tm_id")
  stats = stats_by_pos.loc[stats_by_pos.main_field_position == position]
  stats = stats.loc[~stats_by_pos.attribute.isin(["ppg","mpg"])]
  stats_wide = pd.pivot_table(stats, values = 'value', index = ["tm_id","season"], 
                               columns = "attribute", aggfunc=np.sum)
  stats_wide.columns = stats_wide.columns.astype(list)
  stats_wide.reset_index(inplace = True, drop = False)  
  stats_wide["date"] = stats_wide.season.apply(season_to_date)
  stats_wide["date"] = pd.to_datetime(stats_wide.date, format='%Y', errors = 'coerce')
  return stats_wide

In [0]:
def merge_mean_ppg(stats,position,pos_wide):
  stats_by_pos = pd.merge(stats,
                        players[["tm_id","main_field_position"]],
                        on="tm_id")
  stats = stats_by_pos.loc[stats_by_pos.main_field_position == position]
  stats = stats.loc[stats_by_pos.attribute.isin(["ppg","mpg"])]
  pos_players = players.loc[players.main_field_position == position]
  stats_wide = pd.pivot_table(stats, values = 'value', index = ["tm_id","season"], 
                               columns = "attribute", aggfunc=np.mean)
  stats_wide.columns = stats_wide.columns.astype(list)
  stats_wide.reset_index(inplace = True, drop = False)
  return pd.merge(pos_wide, stats_wide, on = ["tm_id", "season"])

In [0]:
# gk_stats_wide = wide_stats(stats, position = "goalkeepers")

In [0]:
# gk_stats_wide = merge_mean_ppg(stats,"goalkeepers",gk_stats_wide)

In [0]:
def_stats_wide = wide_stats(stats, position = "defenders")
def_stats_wide = merge_mean_ppg(stats,"defenders",def_stats_wide)

In [0]:
def_stats_wide

attribute,tm_id,season,app,a,g,mp,og,pg,rc,syc,s,soff,son,yc,date,mpg,ppg
0,2865,01/02,1.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,2002-01-01,0.00,3.000000
1,2865,02/03,26.0,1.0,0.0,1370.0,0.0,0.0,0.0,0.0,28.0,4.0,12.0,4.0,2003-01-01,0.00,1.060000
2,2865,03/04,34.0,2.0,2.0,2908.0,0.0,0.0,0.0,1.0,35.0,4.0,1.0,10.0,2004-01-01,282.25,1.467500
3,2865,04/05,28.0,1.0,2.0,2375.0,0.0,0.0,0.0,0.0,28.0,4.0,1.0,7.0,2005-01-01,564.00,0.760000
4,2865,05/06,43.0,2.0,2.0,2843.0,0.0,0.0,0.0,0.0,44.0,7.0,12.0,6.0,2006-01-01,197.40,1.372000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46903,48747085,15/16,33.0,0.0,0.0,2380.0,0.0,0.0,0.0,0.0,45.0,2.0,7.0,7.0,2016-01-01,0.00,1.070000
46904,48747085,16/17,35.0,1.0,0.0,2559.0,0.0,0.0,0.0,0.0,46.0,7.0,5.0,16.0,2017-01-01,0.00,1.813333
46905,48747085,17/18,29.0,0.0,0.0,2437.0,1.0,0.0,0.0,0.0,33.0,3.0,1.0,5.0,2018-01-01,0.00,0.555000
46906,48747085,18/19,11.0,0.0,1.0,831.0,0.0,0.0,0.0,0.0,22.0,3.0,1.0,1.0,2019-01-01,370.50,1.750000


In [0]:
mid_stats_wide = wide_stats(stats, position = "midfielders")
mid_stats_wide = merge_mean_ppg(stats,"midfielders",mid_stats_wide)

In [0]:
att_stats_wide = wide_stats(stats, position = "attackers")
att_stats_wide = merge_mean_ppg(stats,"attackers",att_stats_wide)

In [0]:
players_stats_wide = pd.concat([def_stats_wide,mid_stats_wide,att_stats_wide])

In [0]:
players_stats_wide[players_stats_wide.tm_id == 8198]

attribute,tm_id,season,app,a,g,mp,og,pg,rc,syc,s,soff,son,yc,date,mpg,ppg
418,8198,02/03,31.0,1.0,5.0,1391.0,0.0,0.0,0.0,0.0,36.0,10.0,17.0,1.0,2003-01-01,110.500000,1.265000
419,8198,03/04,40.0,9.0,6.0,2355.0,0.0,0.0,0.0,1.0,45.0,12.0,16.0,3.0,2004-01-01,151.500000,1.592500
420,8198,04/05,50.0,10.0,9.0,3716.0,0.0,0.0,0.0,0.0,51.0,16.0,10.0,5.0,2005-01-01,124.200000,1.780000
421,8198,05/06,47.0,9.0,12.0,3413.0,0.0,1.0,1.0,0.0,47.0,15.0,10.0,10.0,2006-01-01,116.600000,1.824000
422,8198,06/07,53.0,21.0,23.0,4386.0,0.0,5.0,0.0,0.0,53.0,14.0,4.0,5.0,2007-01-01,166.750000,1.557500
423,8198,07/08,49.0,8.0,42.0,4154.0,0.0,6.0,1.0,0.0,50.0,6.0,3.0,7.0,2008-01-01,76.500000,2.505000
424,8198,08/09,53.0,12.0,26.0,4319.0,0.0,5.0,0.0,1.0,54.0,9.0,5.0,8.0,2009-01-01,170.000000,2.518000
425,8198,09/10,35.0,12.0,33.0,2915.0,0.0,4.0,1.0,1.0,35.0,5.0,2.0,3.0,2010-01-01,79.500000,2.360000
426,8198,10/11,54.0,18.0,53.0,4618.0,0.0,9.0,0.0,0.0,54.0,5.0,3.0,7.0,2011-01-01,113.666667,2.510000
427,8198,11/12,55.0,15.0,60.0,4902.0,0.0,14.0,0.0,0.0,56.0,3.0,1.0,7.0,2012-01-01,123.000000,1.832500


In [0]:
def cumulative_stats(pos_stats):
  df = pos_stats
  # indexes = ["tm_id","season","date"]
  # cols = pos_stats.columns
  # cols = list(filter(lambda x : x not in indexes, cols))
  cols = ['app', 'cs', 'gc', 'mp', 'og', 'rc', 'syc', 's',
       'soff', 'son', 'yc','a', 'g', 'mpg','og', 'pg','syc','ppg']
  for col in cols:
    if(col in df.columns):
      new_col = "cum_"+ col
      if(col == "ppg" or col == "mpg"):
        df[new_col] = df.groupby("tm_id").apply(lambda df : df[col].rolling(2).mean()).values
      else:
        df[new_col] = df.groupby("tm_id").apply(lambda df : df[col].cumsum() - df[col]).values
  df["cum_games_won"] = (df["cum_ppg"] * df["cum_app"])/3
  # df["cum_games_won"] = df["cum_games_won"].astype("int")
  return df

In [0]:
# gk_cum = cumulative_stats(gk_stats_wide)

In [0]:
players_stats_cum = cumulative_stats(players_stats_wide)

In [0]:
gk_cum

attribute,tm_id,season,app,cs,gc,mp,og,rc,syc,s,soff,son,yc,date,ppg,cum_app,cum_cs,cum_gc,cum_mp,cum_og,cum_rc,cum_syc,cum_s,cum_soff,cum_son,cum_yc,cum_ppg,cum_games_won
0,1013,00/01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,2001-01-01,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0
1,1013,01/02,15.0,4.0,22.0,1291.0,0.0,0.0,0.0,38.0,0.0,1.0,0.0,2002-01-01,1.473333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,1.473333,0
2,1013,02/03,19.0,3.0,21.0,1710.0,0.0,0.0,0.0,24.0,0.0,0.0,0.0,2003-01-01,1.210000,15.0,4.0,22.0,1291.0,0.0,0.0,0.0,47.0,0.0,1.0,0.0,2.683333,13
3,1013,03/04,29.0,15.0,25.0,2584.0,0.0,0.0,0.0,30.0,1.0,0.0,0.0,2004-01-01,1.286667,34.0,7.0,43.0,3001.0,0.0,0.0,0.0,71.0,0.0,1.0,0.0,3.970000,44
4,1013,04/05,19.0,8.0,20.0,1710.0,0.0,0.0,0.0,22.0,0.0,0.0,0.0,2005-01-01,0.840000,63.0,22.0,68.0,5585.0,0.0,0.0,0.0,101.0,1.0,1.0,0.0,4.810000,101
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17011,487487257474,15/16,11.0,4.0,10.0,990.0,0.0,0.0,0.0,12.0,0.0,0.0,2.0,2016-01-01,1.910000,19.0,7.0,22.0,1710.0,0.0,0.0,0.0,20.0,0.0,0.0,3.0,3.600000,22
17012,487487257474,17/18,4.0,1.0,4.0,360.0,0.0,0.0,0.0,37.0,0.0,0.0,0.0,2018-01-01,2.500000,30.0,11.0,32.0,2700.0,0.0,0.0,0.0,32.0,0.0,0.0,5.0,6.100000,61
17013,487487257474,18/19,12.0,3.0,23.0,1065.0,0.0,0.0,1.0,36.0,0.0,0.0,1.0,2019-01-01,1.437500,34.0,12.0,36.0,3060.0,0.0,0.0,0.0,69.0,0.0,0.0,5.0,7.537500,85
17014,487487257474,19/20,3.0,1.0,6.0,270.0,0.0,0.0,0.0,24.0,0.0,0.0,0.0,2020-01-01,0.500000,46.0,15.0,59.0,4125.0,0.0,0.0,1.0,105.0,0.0,0.0,6.0,8.037500,123


In [0]:
gk_cum.to_pickle("/content/drive/My Drive/Capstone/Data/Clean/gk_stats_wide.pkl")

In [0]:
players_stats_cum.to_pickle("/content/drive/My Drive/Capstone/Data/Clean/player_stats_wide.pkl")